### Installation

`pip install tf-encrypted`

In [13]:
import pandas as pd
import logging
# suppress warnings
for name in logging.Logger.manager.loggerDict.keys():
    logging.getLogger(name).setLevel(logging.CRITICAL)
import tensorflow as tf
import tf_encrypted as tfe

Load dataset first in plaintext, in this case the Boston housing price dataset.

In [14]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')
df = pd.read_csv(csv_file)
# Shuffle dataframe
df = df.sample(frac=1)
# Divide into features and target variables
X, y = df[df.columns[:-2]].values, df[df.columns[-1]].values
# Scale features to [0,1]
X = (X - X.min()) / (X.max() - X.min()) * (1 - 0) + 0 
# Divide into training and test data
train_X, train_y, = X[:int(len(X)*0.8)], y[:int(len(X)*0.8)]
test_X, test_y = X[int(len(X)*0.8):], y[int(len(X)*0.8):]

Encrypt dataset using DataOwner class from common.py

In [15]:
from lib.common import DataOwner
num_features = X.shape[1]
training_set_size = int(len(X)*0.8)
test_set_size = int(len(X)*0.2)
batch_size = 4
data_owner = DataOwner(
    "data-owner", num_features, training_set_size, test_set_size, batch_size
)
train_X, train_y = data_owner.provide_training_data(train_X,train_y)
test_X, test_y = data_owner.provide_testing_data(test_X,test_y)

Create a model using Keras and convert to a encrypted model.

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,activation='sigmoid',batch_input_shape=[batch_size, num_features]),
    tf.keras.layers.Dense(10,activation='sigmoid'),
    tf.keras.layers.Dense(1,activation='sigmoid')]
)
encrypted_model = tfe.keras.models.clone_model(model)

In [17]:
from tf_encrypted.keras.losses import BinaryCrossentropy
from tf_encrypted.keras.optimizers import SGD
from tf_encrypted.keras import backend as KE

sess = KE.get_session()
sess.run([data_owner.initializer])
optimizer = SGD(lr=0.01)
loss = BinaryCrossentropy()
encrypted_model.compile(optimizer=optimizer, loss=loss)
encrypted_model.fit(train_X, train_y, epochs=10, steps_per_epoch = training_set_size // batch_size)

Epoch 1/10
240/240 [==============================] - 12s 50ms/step - loss: 0.7564
Epoch 2/10
240/240 [==============================] - 2s 9ms/step - loss: 0.6495
Epoch 3/10
240/240 [==============================] - 2s 10ms/step - loss: 0.5985
Epoch 4/10
240/240 [==============================] - 2s 8ms/step - loss: 0.5772
Epoch 5/10
240/240 [==============================] - 2s 7ms/step - loss: 0.6018
Epoch 6/10
240/240 [==============================] - 2s 7ms/step - loss: 0.5497
Epoch 7/10
240/240 [==============================] - 2s 7ms/step - loss: 0.6123
Epoch 8/10
240/240 [==============================] - 2s 7ms/step - loss: 0.5678
Epoch 9/10
240/240 [==============================] - 2s 7ms/step - loss: 0.5633
Epoch 10/10
240/240 [==============================] - 2s 7ms/step - loss: 0.6013


Use the model to predict on encrypted data

In [18]:
from lib.common import PredictionClient
prediction_client = PredictionClient("prediction-client", num_features)
predictions_encrypted = encrypted_model(test_X)
predictions_plaintext = sess.run(predictions_encrypted.reveal())
predictions_plaintext[0]

array([0.26047859])